In [1]:
import Pkg; 

if split(pwd(),"/")[end] == "notebooks"
    cd(joinpath(@__DIR__, "../"))
    Pkg.activate("Project.toml")
end

using MorphoMol

using JLD2

  Activating project at `~/Desktop/Doktor/MorphoMol/MorphoMolMonteCarlo`


In [20]:
folder = "assets/hpc_assembly/6sag_protor_oj_0_0_os_0_85/"
evaluation_strings = ["" for _ in 1:maximum([parse(Int, split(file, "_")[1]) for file in readdir(folder)])]
for file in readdir(folder)
    @load "$folder$file" in_out_data
    id = parse(Int, split(file, "_")[1])
    string = "$(id): $(in_out_data.output.αs[end]) | $(in_out_data.output.Es[1]), $(minimum(in_out_data.output.Es)), $(in_out_data.output.Es[end])"
    evaluation_strings[id] = string
end

In [21]:
for elem in evaluation_strings
    println(elem)
end

1: 0.4495837 | 8148.573229333724, 8145.822897138247, 8156.690238998561
2: 0.46989837 | 8148.573229333724, 8144.02833090977, 8147.795719981834
3: 0.98844063 | 8148.573229333724, 8148.573229333724, 8174.157936482299
4: 0.37027028 | 8148.573229333724, 8140.528773929401, 8148.326963424011
5: 0.47912526 | 8148.573229333724, 8142.327494478111, 8157.402986078737
6: 0.9881034 | 8148.573229333724, 8148.573229333724, 8174.157936095831
7: 0.11391437 | 8148.573229333724, 8142.182511991475, 8150.2142953469465
8: 0.99131507 | 8148.573229333724, 8147.286546616957, 8174.15793631607
9: 0.9991552 | 8148.573229333724, 8148.573229333724, 8174.157936362723
10: 0.30665162 | 8148.573229333724, 8138.599533728182, 8149.990502801883
11: 0.56586826 | 8148.573229333724, 8144.343583002429, 8145.324709444657
12: 0.33504775 | 8148.573229333724, 8132.145422617905, 8162.366653068462
13: 0.9736196 | 8148.573229333724, 8148.573229333724, 8174.157936046036
14: 0.44761905 | 8148.573229333724, 8141.898205779661, 8159.03429

In [ ]:
MorphoMol.Utilities.state_to_poly(get_flat_realization(state, template_mol), radii, "assets/output/$(i)", n_mol, n_atoms_per_mol)